In [98]:
from lacbox.io import load_stats, load_oper
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

BB_PATH = Path.cwd().parent / 'our_design/BB_turb_stats.csv'  # path to mean steady stats
STATS_PATH = Path.cwd().parent / 'dtu_10MW' / 'turb' / 'dtu_10mw_turb_stats.hdf5'  # path to mean steady stats
SUBFOLDER = 'tca' 

In [99]:
# load the HAWC2 data from the stats file. Isolate the simulations with no tilt.
df, wsps = load_stats(STATS_PATH, subfolder=SUBFOLDER, statstype='turb')
df_BB, wsps_BB = load_stats(BB_PATH, subfolder=SUBFOLDER, statstype='turb')

SEEDS = 6



In [100]:
def extract_and_order_statistics(wsps, chan_df, seeds):
    """
    Extract and order:
     - wsp, max, mean, min for the points
     - wsp, mean_max, mean_mean, mean_min for the mean lines
    """

    # extract hawc2 wind and channel to plot from the HAWC2 stats
    val_wsp, val_max, val_mean, val_min = chan_df['wsp'], chan_df['max'], chan_df['mean'], chan_df['min']
    val_wsp, val_max, val_mean, val_min = np.array(val_wsp), np.array(val_max), np.array(val_mean), np.array(val_min)
    i_val = np.argsort(val_wsp)

    # Get mean of max/mean/min
    mean_max = np.average(val_max[i_val].reshape(-1, seeds), axis=1)
    mean_mean = np.average(val_mean[i_val].reshape(-1, seeds), axis=1)
    mean_min = np.average(val_min[i_val].reshape(-1, seeds), axis=1)

    points = {
        'wsp': val_wsp[i_val],
        'max': val_max[i_val],
        'mean': val_mean[i_val],
        'min': val_min[i_val]
    }

    lines = {
        'wsp': wsps,
        'max': mean_max,
        'mean': mean_mean,
        'min': mean_min
    }

    return points, lines

In [107]:
# isolate the tower clearance channel data
# chan_df = df.filter_channel(113, 'DLL inp   5')
chan_df = df.filter_channel(0, {0 :'DLL :  5 inpvec :   1  min. distance bladetips tower [m]'})

# Filter rows where 'desc' column matches the specific name
filtered_df = df[df['desc'] == 'DLL :  5 inpvec :   1  min. distance bladetips tower [m]']

# Display the filtered dataframe
# print(filtered_df)


tower_clearance = filtered_df['min']
# tower_clearance = chan_df_wtf['min']


tower_clearance_array = np.array(tower_clearance)

print(tower_clearance_array)

print(tower_clearance_array.min())


[11.26050377  8.58535767 13.35652161 15.88511658 11.62287807 14.7456665
  6.63629055  9.70432186  7.31421328 15.27006721  8.53271389  8.11631489
  9.5426693  13.55875397  8.97152424 10.06505871 12.0189867  12.25952244
  9.22528267 12.11332989 14.36962605 11.56655216  8.8756752  12.18406105
 12.74793816  8.55959034 13.33467197  8.73872757 13.02907181 10.95130634
 15.46075344 14.45940685 13.05881882  5.96598768 14.72356415 11.34911346
 10.58613777 13.28807354 14.18466854 13.11685848 15.10324764  8.91434002
  8.48191643 13.01204491  9.86104107 12.22253609  8.42437458 14.47663593
 13.89804745 10.95097446  9.14496899 12.55976772 12.9944849  12.29644871
 15.04215431 12.98379803 13.84835243  7.29614544 11.48008823  8.67921352
 13.38647366  9.54003143 11.8475275  13.50693321  9.64986229 10.46387482
 14.41237926 13.1461544  10.96539879 14.3999052  13.17383385 13.71437645
  8.99560261  7.51823568  9.05344677 13.98763943 14.69485283  9.55340481
 11.51403427 14.81953049 12.43291283 11.12916279 10.

In [109]:
CHAN_DESCS = {'TbFA': 'momentmx mbdy:tower nodenr:   1',
              'TbSS': 'momentmy mbdy:tower nodenr:   1',
              'YbTilt': 'momentmx mbdy:tower nodenr:  11',
              'YbRoll': 'momentmy mbdy:tower nodenr:  11',
              'ShftTrs': 'momentmz mbdy:shaft nodenr:   4',
              'OoPBRM': 'momentmx mbdy:blade1 nodenr:   1 coo: hub1',
              'IPBRM': 'momentmy mbdy:blade1 nodenr:   1 coo: hub1',
              'FlpBRM': 'momentmx mbdy:blade1 nodenr:   1 coo: blade1',
              'EdgBRM': 'momentmy mbdy:blade1 nodenr:   1 coo: blade1',
              'OoPHub': 'momentmx mbdy:hub1 nodenr:   1 coo: hub1',
              'IPHub': 'momentmy mbdy:hub1 nodenr:   1 coo: hub1',
              }

# what channels we want to plot
chan_ids = ['TbFA', 'TbSS',
            'YbTilt', 'YbRoll', 'ShftTrs', 'OoPBRM', 'IPBRM']

max_dict = {}
min_dict = {}
mean_dict = {}
for iplot, chan_id in enumerate(chan_ids):
    
    chan_df = df.filter_channel(chan_id, CHAN_DESCS)
    
    # Doing this works for all channel, but not the tip clearance, because f me
    # channel_wsp = chan_df['wsp']
    # channel_max = chan_df['max']
    # channel_min = chan_df['min']
    # channel_mean = chan_df['mean']
    
    points_BB, lines_BB = extract_and_order_statistics(wsps_BB, chan_df, SEEDS)
    
    max_val = np.array(lines_BB['max']).max()
    min_val = np.array(lines_BB['min']).min()
    mean_val = np.array(lines_BB['mean']).mean()
    
    # max_val = np.array(channel_mean).max()
    max_dict[chan_id]  =  max_val * 1.35 * 1.25
    min_dict[chan_id]  =  min_val * 1.35 * 1.25
    mean_dict[chan_id] = mean_val

print(max_dict)
print(min_dict)
print(mean_dict)


# Initialize the new dictionary for absolute maximum values
absolute_max_dict = {}

# Loop through each key in max_dict (assuming keys in max_dict and min_dict are identical)
for key in max_dict.keys():
    # Calculate the absolute maximum value between max_dict and min_dict for each key
    absolute_max_dict[key] = max(abs(max_dict[key]), abs(min_dict[key]))

# Print the resulting dictionary with absolute maximum values
print(absolute_max_dict)


{'TbFA': 364297.47363281256, 'TbSS': 117720.31530761719, 'YbTilt': 54987.89062500001, 'YbRoll': 23907.457946777344, 'ShftTrs': -685.476744890213, 'OoPBRM': 27557.119445800785, 'IPBRM': 40770.70642089844}
{'TbFA': -51051.62658691406, 'TbSS': -51982.17736816406, 'YbTilt': -38194.008178710945, 'YbRoll': 239.45605301856997, 'ShftTrs': -20365.171325683594, 'OoPBRM': -72275.13830566406, 'IPBRM': -27472.3678894043}
{'TbFA': 96549.52281564193, 'TbSS': 12733.729012465428, 'YbTilt': 1867.128824448829, 'YbRoll': 9102.224890923591, 'ShftTrs': -8842.391787113766, 'OoPBRM': -15499.956871376378, 'IPBRM': 2753.3949561108216}
{'TbFA': 364297.47363281256, 'TbSS': 117720.31530761719, 'YbTilt': 54987.89062500001, 'YbRoll': 23907.457946777344, 'ShftTrs': 20365.171325683594, 'OoPBRM': 72275.13830566406, 'IPBRM': 40770.70642089844}
